In [2]:
# Import findspark and initialize.
import os
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,685 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
sparkSQL = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
sparkSQL.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [51]:
# 2. Create a temporary view of the DataFrame.

sparkSQL.createOrReplaceTempView('home_sales')
spark.sql("SELECT * FROM home_sales").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [52]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
(SELECT round(avg(price),2) as avg_price, year(date) AS year, bedrooms
from home_sales
where bedrooms == 4
group by year, bedrooms
order by year)
"""
spark.sql(query).show()


+---------+----+--------+
|avg_price|year|bedrooms|
+---------+----+--------+
| 300263.7|2019|       4|
|298353.78|2020|       4|
|301819.44|2021|       4|
|296363.88|2022|       4|
+---------+----+--------+



In [53]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
(SELECT round(avg(price),2) as avg_price, date_built, bedrooms, bathrooms
from home_sales
where bedrooms == 3 and bathrooms == 3
group by date_built,bedrooms, bathrooms
order by date_built)
"""
spark.sql(query).show()


+---------+----------+--------+---------+
|avg_price|date_built|bedrooms|bathrooms|
+---------+----------+--------+---------+
|292859.62|      2010|       3|        3|
|291117.47|      2011|       3|        3|
|293683.19|      2012|       3|        3|
|295962.27|      2013|       3|        3|
|290852.27|      2014|       3|        3|
| 288770.3|      2015|       3|        3|
|290555.07|      2016|       3|        3|
|292676.79|      2017|       3|        3|
+---------+----------+--------+---------+



In [54]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
(SELECT round(avg(price),2) as avg_price, date_built, bedrooms, bathrooms, sqft_living
from home_sales
where bedrooms == 3 and bathrooms == 3 and floors == 2 and sqft_living >= 2000
group by date_built,bedrooms, bathrooms, sqft_living
order by date_built)
"""
spark.sql(query).show()


+---------+----------+--------+---------+-----------+
|avg_price|date_built|bedrooms|bathrooms|sqft_living|
+---------+----------+--------+---------+-----------+
| 356911.0|      2010|       3|        3|       2405|
| 287150.0|      2010|       3|        3|       2250|
| 241176.5|      2010|       3|        3|       2300|
| 405100.0|      2010|       3|        3|       2099|
| 139421.0|      2010|       3|        3|       2002|
| 211873.0|      2010|       3|        3|       2218|
| 201255.0|      2010|       3|        3|       2272|
| 424334.0|      2010|       3|        3|       2381|
| 154330.0|      2010|       3|        3|       2183|
| 130941.0|      2010|       3|        3|       2313|
| 193290.0|      2010|       3|        3|       2492|
| 339534.0|      2010|       3|        3|       2139|
| 211041.0|      2010|       3|        3|       2442|
| 133787.0|      2010|       3|        3|       2420|
| 257250.0|      2010|       3|        3|       2040|
| 447889.0|      2010|      

In [55]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
(SELECT round(avg(price),2) as avg_price, view
from home_sales
group by view
having avg(price) >= 350000
order by view desc)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| avg_price|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

--- 0.7444086074829102 seconds ---


In [56]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [57]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [58]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
query = """
(SELECT round(avg(price),2) as avg_price, view
from home_sales
group by view
having avg(price) >= 350000
order by view desc)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----------+----+
| avg_price|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

--- 0.46309423446655273 seconds ---


In [59]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sparkSQL.write.partitionBy('date_built').mode('overwrite').parquet('homes_parquet')

In [60]:
# 11. Read the formatted parquet data.
p_sparkSQl = spark.read.parquet('homes_parquet')

In [61]:
# 12. Create a temporary table for the parquet data.
p_sparkSQl.createOrReplaceTempView("par_homes")

In [62]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
query = """
(SELECT round(avg(price),2) as avg_price, view
from par_homes
group by view
having avg(price) >= 350000
order by view desc)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| avg_price|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

--- 0.5348849296569824 seconds ---


In [63]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [64]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False